# This notebook generates CMG dat files for running CMG simulations

# Step 1: set up a base CMG model
Prepare a base CMG dat file and add it to the wrtcmgdat.py

# Step 2: sample uncertain parameters

## 1. CMG requires initializating stress state using a reference block. For the JD_Sula_2005_gmc grid, there are 10 k layers. Reservoir starts at k=6. Use block (50, 1, 6) as reference block for *STRESSGRAD calculation. Its grid top = 670.7188 m and bottom = 671.9521 m.
## 2. PORO/PERMX pairs are NOT sampled more than once (from file names instead of files)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from pathlib import Path
import sys
import re

# Setup for sampling
random_seed = 11
# Note: 1) stress gradients are effective ones (required by CMG) after subtracting 10; 
#       2) stress gradients are negative due to CMG DIR DOWN convention
params = ['E_GPa', 'PR', 'SH_MPa/km', 'Sh_MPa/km', 'Sv_MPa/km', 'SH_azi_deg']
# l_bounds = [15e6, 0.2, 25 * 0.9, 14.6 * 0.9, 22.7 * 0.9, 290]
# u_bounds = [25e6, 0.4, 25 * 1.1, 14.6 * 1.1, 22.7 * 1.1, 310]
l_bounds = [15e6, 0.2, -15 * 1.1, -4.6 * 1.1, -12.7 * 1.1, 290]
u_bounds = [25e6, 0.4, -15 * 0.9, -4.6 * 0.9, -12.7 * 0.9, 310]
num_samples = 90  # Change as needed

# Load PORO and PERMX file names
# property_file_names = np.load('property_file_names.npy')
property_file_names = np.loadtxt("property_file_names.csv",delimiter=",",dtype=str)

# sort the file names by the number in the name
def extract_number(filename):
    match = re.search(r"(\d+)", filename)
    return int(match.group(1)) if match else float('inf')

poro_file_names = sorted(
    [name for name in property_file_names if "PORO" in name.upper()],
    key=extract_number
)

permx_file_names = sorted(
    [name for name in property_file_names if "PERMX" in name.upper()],
    key=extract_number
)

# check a few things
if not poro_file_names or not permx_file_names:
    print("Error: PORO or PERMX file names not found.")
    sys.exit(1)

if len(poro_file_names) != len(permx_file_names):
    raise ValueError(f"Number of PORO file names ({len(poro_file_names)}) does not match number of PERMX file names ({len(permx_file_names)})")

num_pairs = len(poro_file_names)

if num_samples > num_pairs:
    raise ValueError(f"Cannot sample {num_samples} unique poro/permx pairs: only {num_pairs} available.")

# Latin Hypercube Sampling for parameters
sampler = qmc.LatinHypercube(d=len(params), seed=random_seed)
sample = sampler.random(n=num_samples)
sample_params = qmc.scale(sample, l_bounds, u_bounds)
# df_params = pd.DataFrame(np.round(sample_params, 2), columns=params)
df_params = pd.DataFrame(sample_params, columns=params)

# Store poro/permx pairs
df_params["PORO_file"] = [str(poro_file_names[i]) for i in range(num_samples)]
df_params["PERMX_file"] = [str(permx_file_names[i]) for i in range(num_samples)]

# add prefix to file names
prefix = "data_properties/"
df_params["PORO_file"] = df_params["PORO_file"].apply(lambda x: f"{prefix}{x}")
df_params["PERMX_file"] = df_params["PERMX_file"].apply(lambda x: f"{prefix}{x}")

# Calculate stress state parameters
df_params['beta'] = df_params['SH_azi_deg'] - 90  # Rotate from SH to x-axis
df_params['cos_2beta'] = np.cos(np.radians(2 * df_params['beta']))
df_params['sin_2beta'] = np.sin(np.radians(2 * df_params['beta']))
# calculate the stress gradients in kPa/km
df_params['sigma_x_grad'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 + \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
df_params['sigma_y_grad'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 - \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
# tau_xy_grad should be positive after checking the directions of maximum stress in the CMG Results
df_params['tau_xy_grad'] = -(df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['sin_2beta']
# calculate the stress state for the reference block in kPa
# for the JD_Sula_2005_gmc grid, the reference block is (50, 1, 6) 
grid_top = 670.7188; grid_bottom = 671.9521; grid_ave = (grid_top + grid_bottom)/2
df_params['sigma_x_ref'] = df_params['sigma_x_grad'] * grid_ave *(-1)
df_params['sigma_y_ref'] = df_params['sigma_y_grad'] * grid_ave *(-1)
df_params['sigma_z_ref'] = df_params['Sv_MPa/km'] * grid_ave *(-1)
df_params['tau_xy_ref'] = df_params['tau_xy_grad'] * grid_ave *(-1)

# Output
df_params.to_csv("sampled_parameters_and_files.csv", index=False,float_format='%.2f')
df_params.round(2)


,E_GPa,PR,SH_MPa/km,Sh_MPa/km,Sv_MPa/km,SH_azi_deg,PORO_file,PERMX_file,beta,cos_2beta,sin_2beta,sigma_x_grad,sigma_y_grad,tau_xy_grad,sigma_x_ref,sigma_y_ref,sigma_z_ref,tau_xy_ref
0,17430158.87,0.32,-15.82,-4.71,-12.37,296.90,data_properties/JD_BASECASE_5_PORO.dat,data_properties/JD_BASECASE_5_PERMX.dat,206.90,0.59,0.81,-13.55,-6.99,4.48,9093.68,4690.71,8301.40,-3009.02
1,18547731.05,0.23,-14.96,-4.30,-12.80,300.55,data_properties/JD_BASECASE_6_PORO.dat,data_properties/JD_BASECASE_6_PERMX.dat,210.55,0.48,0.88,-12.21,-7.06,4.67,8196.33,4736.70,8589.79,-3134.33
2,21148573.01,0.27,-14.37,-4.63,-12.92,308.11,data_properties/JD_BASECASE_7_PORO.dat,data_properties/JD_BASECASE_7_PERMX.dat,218.11,0.24,0.97,-10.66,-8.34,4.73,7156.76,5598.45,8671.29,-3176.16
3,15242584.84,0.31,-13.70,-4.34,-11.98,297.23,data_properties/JD_BASECASE_8_PORO.dat,data_properties/JD_BASECASE_8_PERMX.dat,207.23,0.58,0.81,-11.74,-6.30,3.81,7881.23,4226.76,8043.84,-2557.12
4,18738525.02,0.32,-15.67,-4.52,-13.77,291.53,data_properties/JD_BASECASE_9_PORO.dat,data_properties/JD_BASECASE_9_PERMX.dat,201.53,0.73,0.68,-14.17,-6.02,3.81,9514.24,4040.50,9243.42,-2556.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,23335499.95,0.31,-15.57,-4.43,-12.77,301.18,data_properties/JD_BASECASE_194_PORO.dat,data_properties/JD_BASECASE_194_PERMX.dat,211.18,0.46,0.89,-12.59,-7.42,4.93,8449.42,4979.71,8575.10,-3312.66
86,24592116.34,0.30,-15.92,-4.19,-12.21,291.78,data_properties/JD_BASECASE_197_PORO.dat,data_properties/JD_BASECASE_197_PERMX.dat,201.78,0.72,0.69,-14.31,-5.81,4.04,9603.99,3897.65,8197.62,-2713.86
87,16276499.15,0.30,-16.40,-4.24,-13.53,302.25,data_properties/JD_BASECASE_198_PORO.dat,data_properties/JD_BASECASE_198_PERMX.dat,212.25,0.43,0.90,-12.94,-7.70,5.49,8687.92,5172.36,9083.68,-3685.14
88,20519907.95,0.39,-15.63,-4.44,-12.94,297.56,data_properties/JD_BASECASE_199_PORO.dat,data_properties/JD_BASECASE_199_PERMX.dat,207.56,0.57,0.82,-13.23,-6.84,4.59,8884.20,4590.81,8689.28,-3079.98


In [24]:
np.savetxt("property_file_names.csv",property_file_names,delimiter=",",fmt="%s")
property_file_names_txt = np.loadtxt("property_file_names.csv",delimiter=",",dtype=str)
property_file_names_txt.shape

(180,)

# Step3: generate CMG dat files based on the sampled parameters

In [22]:
import pandas as pd
from generate_dat_files import generate_dat_files

df_params = pd.read_csv('sampled_parameters_and_files.csv')
generate_dat_files(
    df_parameters = df_params,
    template_file_path = "dat_file_templates/250913.dat",
    save_folder_path = "dat_files"
)

Generated 90 dat files successfully.


# Step 4: run CMG simulations

## Option 1: on TS machines

In [7]:
# Record the start time
start_time = time.time()

from utils.pyCMG_Control import pycmgcontrol

for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10', 'ese-ts2win-v2024.20'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.run_gem_simulation(case_name_suffix=f'case{nn+1}.dat')
    
# Record the end time
end_time = time.time()

# Calculate the running time
elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

Elapsed time: 52.20 minutes


## Option 2: on Sherlock using the generated dat files